# HW03
2021-12659 박유나

## Import the library

In [ ]:
!pip install evaluate

In [2]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

In [3]:
import pandas as pd

import torch

from datasets import load_dataset # for downloading the dataset

from transformers import BertForSequenceClassification, BertTokenizer # for model and tokenizer
from transformers import DataCollatorWithPadding # for padding
from transformers import AdamW # for optimizing
from transformers import get_scheduler # for checking the training step

from torch.utils.data import DataLoader # for building the Dataloader

## Setting the model, tokenizer and preprocessing the dataset

In [4]:
# Download the data
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
# Check the data
pd.DataFrame(dataset['train'])

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
8525,any enjoyment will be hinge from a personal th...,0
8526,if legendary shlockmeister ed wood had ever ma...,0
8527,hardly a nuanced portrait of a young woman's b...,0
8528,"interminably bleak , to say nothing of boring .",0


In [6]:
# Loading a pre_trained Bert based model
model = BertForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels = 2)
# Loading a tokenizer
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

### 모델 선택 이유
`bert-base-uncased` 모델을 선택하였다.
모델 선택 이유는 첫째로는 버트베이스 모델이기 때문이며, 둘째로는 데이터가 영어로만 이루어져있기 때문에 영어코퍼스로 학습한 모델을 택했다. 셋째로는 해당 모델은 대규모 텍스트 말뭉치에 대해 사전 훈련된 상태로 제공되기 때문에 파인튜닝만한다면 Task의 목적을 달성할 수 있을 것이기 때문이다. 마지막으로는 Task가 sentiment classification이기에 이를 위해 head로 `ForSequenceClassfication`을 사용했다. 그리고 대소문자는 문장분류에 중요하지 않을 것이라 생각해 input을 모두 소문자로 바꾸는 `uncased` 모델을 사용했다.


In [7]:
#Tokeninzing
def tokenize_function(dataset):
  return tokenizer(dataset["text"],padding=True,truncation=True,)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [8]:
pd.DataFrame(tokenized_dataset['train']).head()

,text,label,input_ids,token_type_ids,attention_mask
0,the rock is destined to be the 21st century's ...,1,"[101, 1996, 2600, 2003, 16036, 2000, 2022, 199...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"the gorgeously elaborate continuation of "" the...",1,"[101, 1996, 9882, 2135, 9603, 13633, 1997, 100...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,effective but too-tepid biopic,1,"[101, 4621, 2021, 2205, 1011, 8915, 23267, 160...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
3,if you sometimes like to go to the movies to h...,1,"[101, 2065, 2017, 2823, 2066, 2000, 2175, 2000...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"emerges as something rare , an issue movie tha...",1,"[101, 19391, 2004, 2242, 4678, 1010, 2019, 327...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [9]:
# Data_collator
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [10]:
# Removing useless column
tokenized_datasets = tokenized_dataset.remove_columns(['text'])
tokenized_datasets.set_format("torch")

In [12]:
# Building the Data Loader
train_dataloader = DataLoader(
    tokenized_datasets['train'], shuffle = True, batch_size = 16, collate_fn = data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets['test'], shuffle = True, batch_size = 16, collate_fn = data_collator
)

In [13]:
# Setting the optimizer
optimizer = AdamW(model.parameters(), lr = 5e-5)

#  Setting the learning rate scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)
print(num_training_steps)

1602


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

## Training and evaluate

In [15]:
# Training
from tqdm.auto import tqdm # for checking the progress

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k : v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)


  0%|          | 0/1602 [00:00<?, ?it/s]

In [27]:
# Evaluating
import evaluate

m1 = evaluate.load("f1")
m2 = evaluate.load("accuracy")

model.eval()
for batch in eval_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim= -1)

  m1.add_batch(predictions = predictions, references = batch["labels"])
  m2.add_batch(predictions = predictions, references = batch["labels"])

f1 = m1.compute()
acc = m2.compute()

In [30]:
model.save_pretrained("C:\Workspace\20241\CL")

## Accuracy and F1 score

In [28]:
print("Accuracy:", acc['accuracy'], "F1:", f1['f1'])

Accuracy: 0.8536585365853658 F1: 0.8550185873605948


### Explain the result

해당 모델은 영화리뷰에 대한 감정분석 태스크에서 약 85.37%의 정확도와 약 85.50%의 F1 점수를 얻었다. 정확도는 전체 테스트 셋 중 올바르게 예측된 비율이라  테스트 세트의 약 85.37%가 맞게 분류되었음을 의미한다. F1 점수는 Precision(모델이 positive로 판단한 것 중 실제 positive의 비율) Recall(실제 positive 중 모델에 positivie로 판단한 비율)의 조화 평균입니다. 이 점수는 1에 가까울 수록 분류 성능이 좋다. 따라서 0.85라는 점수는 모델이 분류를 잘할 뿐아니라 recall과 precision사이의 균형이 잡혔다. 즉, 모델이 긍정 부정에 대해 편향적으로 분류하지 않았다고도 볼 수 있다.